Definimos el número de usuarios e ítems:

In [14]:
from utils import procesar_dataframe
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense, Concatenate
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, f1_score, ndcg_score
ratings, test_ratings, NUM_ITEMS, NUM_USERS = procesar_dataframe('./DATOS/datasets_clase/All_Beauty.csv', 1, 5)

Cargamos los votos de entrenamiento. La carga de estos datos, por imposición de `keras` no se hace en una matriz como en los modelos de factorización matricial. Se generan dos *arrays* con los códigos de los usuarios y los ítems y un tercer *array* con las votaciones:

In [15]:
X_train = [np.array([], dtype=int), np.array([], dtype=int)]
y_train = np.array([], dtype=int)

for u in range(len(ratings)):
  for i in range(len(ratings[u])):
    if ratings[u][i] != None:
        X_train[0] = np.append(X_train[0], int(u))
        X_train[1] = np.append(X_train[1], int(i))
        y_train = np.append(y_train, int(ratings[u][i]))

Cargamos también los votos de test del mismo modo:

In [16]:
X_test = [np.array([], dtype=int), np.array([], dtype=int)]
y_test = np.array([], dtype=int)

for u in range(len(test_ratings)):
  for i in range(len(test_ratings[u])):
    if test_ratings[u][i] != None:
        X_test[0] = np.append(X_test[0], int(u))
        X_test[1] = np.append(X_test[1], int(i))
        y_test = np.append(y_test, int(test_ratings[u][i]))

Los hiper-parámetros de nuestro modelo serán el número de factores latentes (`latent_dim`) y el número de iteraciones del entrenamiento (`epochs`).

In [17]:
latent_dim = 5
epochs = 10

Definimos la arquitectura, en este caso con dos capas densas para el MLP de 20 y 10 neuronas:

In [18]:
user_input = Input(shape=[1])
user_embedding = Embedding(NUM_USERS, latent_dim)(user_input)
user_vec = Flatten()(user_embedding)

item_input = Input(shape=[1])
item_embedding = Embedding(NUM_ITEMS, latent_dim)(item_input)
item_vec = Flatten()(item_embedding)

concat = Concatenate(axis=1)([user_vec, item_vec])
d1 = Dense(20, activation='relu')(concat)
d2 = Dense(10, activation='relu')(d1)
output = Dense(1, activation='relu')(d2)

MLP = Model([user_input, item_input], output)

Compilamos y entrenamos el modelo:

In [19]:
MLP.compile(optimizer='adam', metrics=['mae'], loss='mean_squared_error')
MLP.summary()
MLP.fit(X_train, y_train, epochs=epochs, verbose=1)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 5)      │      3,010 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 5)      │      5,390 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 5)         │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 5)         │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 10)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 20)        │        220 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 10)        │        210 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         11 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,841 (34.54 KB)

 Trainable params: 8,841 (34.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 912us/step - loss: 19.8525 - mae: 4.3705
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.7629 - mae: 1.3940
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - loss: 0.5404 - mae: 0.5254
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - loss: 0.3511 - mae: 0.3726
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - loss: 0.2795 - mae: 0.3117
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - loss: 0.2016 - mae: 0.2563
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - loss: 0.1762 - mae: 0.2325
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - loss: 0.1270 - mae: 0.1893
Epoch 9/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - loss: 0.1173 - mae: 0.1717
Epoch 10/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 0.1018 - mae: 0.1529


Calculamos las predicciones:

In [20]:
y_pred = MLP.predict(X_test)
y_pred = np.round(y_pred).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


Medimos el error:

In [21]:
# Métricas de clasificación
print("Precision Score:", precision_score(y_test, y_pred.round(), average='weighted'))
print("Recall Score:", recall_score(y_test, y_pred.round(), average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred.round(), average='weighted'))


Precision Score: 0.9037928603101241
Recall Score: 0.8913551401869159
F1 Score: 0.8940844542866618


/home/pandita/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# Calcular NDCG para cada clase y promediar los resultados
y_pred_classes = np.argmax(y_pred, axis=1)
# Calcular NDCG para cada clase y promediar los resultados
ndcg_scores = []
for class_label in range(6):
    y_test_class = (y_test == class_label).astype(int)
    y_pred_class = (y_pred_classes == class_label).astype(int)
    ndcg_class = ndcg_score([y_test_class], [y_pred_class])
    ndcg_scores.append(ndcg_class)

# Calcular el promedio de los NDCG scores para todas las clases
average_ndcg = np.mean(ndcg_scores)

print("Average NDCG Score:", average_ndcg)

Average NDCG Score: 0.46239428642144254


In [23]:
print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))


0.11448598130841121
0.1261682242990654


## Referencias

He, X., Liao, L., Zhang, H., Nie, L., Hu, X., & Chua, T. S. (2017, April). **Neural collaborative filtering**. In Proceedings of the 26th international conference on world wide web (pp. 173-182).
